# Aprendizado de IA (Inteligência Artificial)


#### Este Jupyter Notebook tem a finalidade de servir como ferramenta pedagógica para o aprendizado de Inteligência Artificial.

# --------------------------------------------------------------------------

### Nível 1 - Atividade 1
##### ● Criar perfis diários sobre a geração fotovoltaica utilizando como variável principal a POTÊNCIA (W);
##### ● Gerar previsão inicial com RNA (Redes Neurais Artificiais);
##### ● Aplicar essa previsão inicial na técnica Monte Carlo Dropout;
##### ● Analisar o que foi obtido dessas atividades.

# --------------------------------------------------------------------------

#### Criando perfis diários para dados climáticos

In [19]:
## Importando as bibliotecas necessárias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
#import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [20]:
## Tratando os dados horários da NASA

data_hourly = pd.read_csv('Database/dados_clima_2018_2021.csv', sep=',',header=0)

data_hourly['Date'] = data_hourly[['DY', 'MO', 'YEAR', 'HR']].apply(lambda x : datetime(year=x[2],month=x[1],day=x[0],hour=x[3]), axis=1)
data_hourly.drop(['DY', 'MO', 'YEAR', 'HR', 'SZA', 'PRECTOTCORR', 'WS10M', 'WD10M'], axis=1, inplace=True)

data_hourly.set_index('Date', inplace=True)
data_hourly.replace(-999, 0, inplace=True)

data_hourly.rename(columns={'ALLSKY_KT':  'kt',
                   'T2M': 'Temperatura a 2 metros (°C)',
                   'RH2M': 'Umidade Relativa a 2 metros (%)',
                   'CLRSKY_SFC_SW_DWN': 'Irradiância Global Horizontal a Céu Claro(kWh/m²)',
                   'ALLSKY_SFC_SW_DWN': 'Irradiância Global Horizontal para Qualquer Céu (kWh/m^2)'}, inplace=True)

data_hourly.drop(data_hourly.index[22530:22536], axis=0, inplace=True)
data_hourly


,kt,Irradiância Global Horizontal a Céu Claro(kWh/m²),Irradiância Global Horizontal para Qualquer Céu (kWh/m^2),Temperatura a 2 metros (°C),Umidade Relativa a 2 metros (%)
Date,,,,,
2018-08-01 00:00:00,0.00,0.00,0.00,24.53,71.69
2018-08-01 01:00:00,0.00,0.00,0.00,24.13,73.38
2018-08-01 02:00:00,0.00,0.00,0.00,23.84,74.44
2018-08-01 03:00:00,0.00,0.00,0.00,23.64,75.12
2018-08-01 04:00:00,0.00,0.00,0.00,23.49,75.56
...,...,...,...,...,...
2021-02-24 13:00:00,0.61,999.90,825.93,32.61,45.44
2021-02-24 14:00:00,0.61,896.90,759.57,32.76,44.19
2021-02-24 15:00:00,0.56,724.05,580.98,32.55,44.25


In [21]:
trace1 = go.Scatter(x = data_hourly.index[0:24], # eixo x = meses da lista 'date
                    y = data_hourly['Irradiância Global Horizontal a Céu Claro(kWh/m²)'][504:528], # eixo y = temperatura máxima
                    mode = 'markers+lines', # tipo do gráfico: marcadorms em linhas
                    name = 'Energia por hora') # legenda do objeto trace: T.MAX_INMET
    
# cria uma lista com os trace's
data = [trace1]

# cria um objeto layout para por em um gráfico
# objeto layout: informar o que está sendo apresentado no gráfico, no caso abaixo, no eixo y é a 'Temperatura °C'
# no eixo x é informado que fica vazio, com o título do gráfico em caixa alta - comando .upper()
# alterando as características da fonte e cor de cada trace do gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Irradiância Global'},xaxis={'title': 'Hora'}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#FF0000', '#00FF7F','#006400', '#000080', '#B22222', '#00BFFF'])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%H:%M',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 1 - Irradiância Global Horizontal a Céu Claro(kWh/m²) no dia 22/8/2018",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix=" kWh/m²") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show()

In [22]:
trace1 = go.Scatter(x = data_hourly.index[0:24], # eixo x 
                    y = data_hourly['Irradiância Global Horizontal a Céu Claro(kWh/m²)'][504:528], # eixo y
                    mode = 'markers+lines', # tipo do gráfico: marcador em linhas
                    name = 'Irradiância Global Horizontal a Céu Claro(kWh/m²)') # legenda do objeto trace

trace2 = go.Scatter(x = data_hourly.index[0:24], # eixo x 
                    y = data_hourly['Irradiância Global Horizontal para Qualquer Céu (kWh/m^2)'][504:528], # eixo y
                    mode = 'markers+lines', # tipo do gráfico: marcador em linhas
                    name = 'Irradiância Global Horizontal para Qualquer Céu (kWh/m^2)') # legenda do objeto trace

# cria uma lista com os trace's
data = [trace1, trace2]

# cria um objeto layout para por em um gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Irradiância Global'},xaxis={'title': 'Hora'}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#FF0000', '#006400', '#00FF7F','#006400', '#000080', '#B22222', '#00BFFF'])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%H:%M',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 2 - Irradiância Global Horizontal a Céu Claro vs Qualquer Céu no dia 22/8/2018",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix=" kWh/m²") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show()

In [23]:
## Tratando os dados de produção fotovoltaica

data_photovoltaic = pd.read_csv('Database\Dados_FV_2018_2021.csv', sep=",")
data_photovoltaic.drop([0],inplace=True)
data_photovoltaic.drop(['Corrente CA L1|Primo 3.0-1 (# 1)', 'Corrente CC MPP1|Primo 3.0-1 (# 1)', 'Energia|Primo 3.0-1 (# 1)', 'Potência aparente|Primo 3.0-1 (# 1)', 'Voltagem CA L1|Primo 3.0-1 (# 1)', 'Voltagem CC MPP1|Primo 3.0-1 (# 1)'], axis=1, inplace=True)
data_photovoltaic.rename(columns={'Data e horário':  'Date',
                   'Produção fotovoltaica': 'Energia (Wh)'}, inplace=True)
data_photovoltaic['Date'] = pd.to_datetime(data_photovoltaic['Date'], format= '%d.%m.%Y %H:%M')
data_photovoltaic['Energia (Wh)'] = data_photovoltaic['Energia (Wh)'].str.replace(',', '.', regex=True).astype(float) 
data_photovoltaic.set_index('Date', inplace=True)
data_photovoltaic = data_photovoltaic.resample('H').sum()

data_photovoltaic

,Energia (Wh)
Date,
2018-08-01 00:00:00,0.00
2018-08-01 01:00:00,0.00
2018-08-01 02:00:00,0.00
2018-08-01 03:00:00,0.00
2018-08-01 04:00:00,0.00
...,...
2021-02-24 13:00:00,1679.89
2021-02-24 14:00:00,1104.89
2021-02-24 15:00:00,481.93


In [24]:
trace1 = go.Scatter(x = data_photovoltaic.index[0:24], # eixo x
                    y = data_photovoltaic['Energia (Wh)'][504:528], # eixo y 
                    mode = 'markers+lines', # tipo do gráfico: marcadr em linhas
                    name = 'Energia por hora') # legenda do objeto trace
    
# cria uma lista com os trace's
data = [trace1]

# cria um objeto layout para por em um gráfico
# objeto layout: informar o que está sendo apresentado no gráfico
# no eixo x é informado que fica vazio, com o título do gráfico em caixa alta - comando .upper()
# alterando as características da fonte e cor de cada trace do gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Irradiância Global'},xaxis={'title': 'Hora'}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#00FF7F', '#FF0000', '#006400', '#000080', '#B22222', '#00BFFF'])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%H:%M',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 3 - Produção de energia fotovoltaica no dia 22/8/2018",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix=" Wh") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show()

In [25]:
## Juntando os dados da NASA e produção de energia fotovoltaica

data_hourly = data_hourly.join(data_photovoltaic)
data_hourly

,kt,Irradiância Global Horizontal a Céu Claro(kWh/m²),Irradiância Global Horizontal para Qualquer Céu (kWh/m^2),Temperatura a 2 metros (°C),Umidade Relativa a 2 metros (%),Energia (Wh)
Date,,,,,,
2018-08-01 00:00:00,0.00,0.00,0.00,24.53,71.69,0.00
2018-08-01 01:00:00,0.00,0.00,0.00,24.13,73.38,0.00
2018-08-01 02:00:00,0.00,0.00,0.00,23.84,74.44,0.00
2018-08-01 03:00:00,0.00,0.00,0.00,23.64,75.12,0.00
2018-08-01 04:00:00,0.00,0.00,0.00,23.49,75.56,0.00
...,...,...,...,...,...,...
2021-02-24 13:00:00,0.61,999.90,825.93,32.61,45.44,1679.89
2021-02-24 14:00:00,0.61,896.90,759.57,32.76,44.19,1104.89
2021-02-24 15:00:00,0.56,724.05,580.98,32.55,44.25,481.93


In [26]:
trace1 = go.Scatter(x = data_hourly.index[0:24], # eixo x 
                    y = data_hourly['Irradiância Global Horizontal a Céu Claro(kWh/m²)'][504:528], # eixo y
                    mode = 'markers+lines', # tipo do gráfico: marcador em linhas
                    name = 'Irradiância Global Horizontal a Céu Claro(kWh/m²)') # legenda do objeto trace

trace2 = go.Scatter(x = data_hourly.index[0:24], # eixo x 
                    y = data_hourly['Irradiância Global Horizontal para Qualquer Céu (kWh/m^2)'][504:528], # eixo y
                    mode = 'markers+lines', # tipo do gráfico: marcador em linhas
                    name = 'Irradiância Global Horizontal para Qualquer Céu (kWh/m^2)') # legenda do objeto trace

trace3 = go.Scatter(x = data_hourly.index[0:24], # eixo x 
                    y = data_hourly['Energia (Wh)'][504:528], # eixo y
                    mode = 'markers+lines', # tipo do gráfico: marcador em linhas
                    name = 'Produção de Energia Fotovoltaica (Wh)') # legenda do objeto trace

# cria uma lista com os trace's
data = [trace1, trace2, trace3]

# cria um objeto layout para por em um gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Energia'},xaxis={'title': 'Hora'}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#FF0000', '#006400', '#000080', '#00FF7F','#006400', '#B22222', '#00BFFF'])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%H:%M',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 4 - Irradiância Global Horizontal a Céu Claro vs Qualquer Céu no dia 22/8/2018",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix="") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show()

In [27]:
## Transformando dados horários em diários 

data_daily_part1 = data_hourly[data_hourly.columns[[0, 3, 4]]].resample('D').mean()
data_daily_part2 = data_hourly[data_hourly.columns[[1, 2, 5]]].resample('D').sum()
data_daily = pd.concat([data_daily_part1, data_daily_part2], axis=1)
data_daily

,kt,Temperatura a 2 metros (°C),Umidade Relativa a 2 metros (%),Irradiância Global Horizontal a Céu Claro(kWh/m²),Irradiância Global Horizontal para Qualquer Céu (kWh/m^2),Energia (Wh)
Date,,,,,,
2018-08-01,0.288750,28.672083,53.761250,6415.55,5638.79,15031.41
2018-08-02,0.297500,28.487500,50.562083,6424.21,5678.10,12754.93
2018-08-03,0.338750,27.872917,46.375000,6683.66,6651.71,16581.87
2018-08-04,0.314167,28.124583,47.484167,6641.93,6156.04,15573.27
2018-08-05,0.328750,29.085000,50.660000,6594.66,6491.30,16056.71
...,...,...,...,...,...,...
2021-02-20,0.307083,28.108333,66.469583,7491.07,6674.68,14190.33
2021-02-21,0.286667,29.076250,61.997500,7573.26,6237.11,13645.42
2021-02-22,0.336250,29.329583,58.138333,7705.41,7431.68,16692.07


In [28]:
trace1 = go.Scatter(x = data_daily.index[0:24], # eixo x
                    y = data_daily['Energia (Wh)'][504:528], # eixo y 
                    mode = 'markers+lines', # tipo do gráfico: marcadr em linhas
                    name = 'Energia por hora') # legenda do objeto trace
    
# cria uma lista com os trace's
data = [trace1]

# cria um objeto layout para por em um gráfico
# objeto layout: informar o que está sendo apresentado no gráfico
# no eixo x é informado que fica vazio, com o título do gráfico em caixa alta - comando .upper()
# alterando as características da fonte e cor de cada trace do gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Energia produzida'},xaxis={'title': 'Hora'}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#B22222', '#00FF7F', '#FF0000', '#006400', '#000080', '#00BFFF'])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%x',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 5 - Produção de energia fotovoltaica entre 2018 e 2021",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix=" Wh") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show()

In [29]:
def data_proc(data, n_data_before, n_data_after):

  for i in range(1, n_data_before + 1):
    data['kt (k-'+str(i)+')'] = data['kt'].shift(i)
    data['Temperatura a 2 metros (°C) (k-'+str(i)+')'] = data['Temperatura a 2 metros (°C)'].shift(i)
    data['Umidade Relativa a 2 metros (%) (k-'+str(i)+')'] = data['Umidade Relativa a 2 metros (%)'].shift(i)
    data['Energia (Wh) (k-'+str(i)+')'] = data['Energia (Wh)'].shift(i)

  for k in range(1, n_data_after + 1):
    data['Energia (Wh) (k+'+str(k)+')'] = data['Energia (Wh)'].shift(k)
        
  return data

In [30]:
n_data_before = 7
n_data_after = 1

data_daily_k = data_proc(data_daily , n_data_before, n_data_after)

data_daily_k.dropna(axis=0,inplace=True)
data_daily_k

,kt,Temperatura a 2 metros (°C),Umidade Relativa a 2 metros (%),Irradiância Global Horizontal a Céu Claro(kWh/m²),Irradiância Global Horizontal para Qualquer Céu (kWh/m^2),Energia (Wh),kt (k-1),Temperatura a 2 metros (°C) (k-1),Umidade Relativa a 2 metros (%) (k-1),Energia (Wh) (k-1),...,Energia (Wh) (k-5),kt (k-6),Temperatura a 2 metros (°C) (k-6),Umidade Relativa a 2 metros (%) (k-6),Energia (Wh) (k-6),kt (k-7),Temperatura a 2 metros (°C) (k-7),Umidade Relativa a 2 metros (%) (k-7),Energia (Wh) (k-7),Energia (Wh) (k+1)
Date,,,,,,,,,,,,,,,,,,,,,
2018-08-08,0.341667,28.335000,49.058750,6738.91,6728.41,16406.57,0.334167,28.723750,49.219583,15812.41,...,16581.87,0.297500,28.487500,50.562083,12754.93,0.288750,28.672083,53.761250,15031.41,15812.41
2018-08-09,0.330417,28.068750,53.166667,6697.75,6138.11,15215.39,0.341667,28.335000,49.058750,16406.57,...,15573.27,0.338750,27.872917,46.375000,16581.87,0.297500,28.487500,50.562083,12754.93,16406.57
2018-08-10,0.349583,28.515833,51.335833,6738.48,6518.86,15634.63,0.330417,28.068750,53.166667,15215.39,...,16056.71,0.314167,28.124583,47.484167,15573.27,0.338750,27.872917,46.375000,16581.87,15215.39
2018-08-11,0.360000,28.135833,47.601250,6799.00,6738.63,15757.18,0.349583,28.515833,51.335833,15634.63,...,16279.71,0.328750,29.085000,50.660000,16056.71,0.314167,28.124583,47.484167,15573.27,15634.63
2018-08-12,0.338333,28.613750,55.525417,6611.71,6379.75,15747.87,0.360000,28.135833,47.601250,15757.18,...,15812.41,0.338333,28.825833,49.395833,16279.71,0.328750,29.085000,50.660000,16056.71,15757.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-20,0.307083,28.108333,66.469583,7491.07,6674.68,14190.33,0.267917,26.849167,67.754583,12527.12,...,10585.29,0.265833,29.872500,55.341667,14751.13,0.292500,30.877083,53.234167,13325.60,12527.12
2021-02-21,0.286667,29.076250,61.997500,7573.26,6237.11,13645.42,0.307083,28.108333,66.469583,14190.33,...,8072.61,0.229167,27.113333,73.804167,10585.29,0.265833,29.872500,55.341667,14751.13,14190.33
2021-02-22,0.336250,29.329583,58.138333,7705.41,7431.68,16692.07,0.286667,29.076250,61.997500,13645.42,...,14344.80,0.157917,27.073333,76.388750,8072.61,0.229167,27.113333,73.804167,10585.29,13645.42


In [36]:
#implementação da RNA

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.layers import LSTM
from keras.layers import Dense

In [37]:
## Funções para normalizar e desnormalizar os dados 

def Normalize(data):
    data_max = data.max(axis=0)
    data_min = data.min(axis=0)
    return (data-data_min)/(data_max-data_min)
    # w = np.sqrt((data**2).sum(axis=0))
    # x_norm2 = data/w
    # return x_norm2

def Denormalize(data_normalized, data):
    data_max = data.max(axis=0)
    data_min = data.min(axis=0)
    return data_normalized*(data_max-data_min) + data_min
    # w = np.sqrt((data**2).sum(axis=0))
    # data_denormalized = data_normalized*w
    # return data_denormalized

In [38]:
## Criando dados de treinamento e testes

inputs = data_daily_k.iloc[:,0:-n_data_after]
outputs = data_daily_k.iloc[:,-n_data_after:]

#normalização
data_norm = Normalize(data_daily_k)

data_daily_norm = data_proc(data_norm, n_data_before, n_data_after)

data_daily_norm.dropna(axis=0,inplace=True)

data_norm = data_daily_norm

inputs_norm = data_norm.iloc[:, 0:-n_data_after]
outputs_norm = data_norm.iloc[:, -n_data_after:]

input_train = data_norm.iloc[0:int(0.7 * len(inputs_norm)), 0:-n_data_after]
input_test = data_norm.iloc[int(0.7 * len(inputs_norm)):-1, 0:-n_data_after] 

output_train = data_norm.iloc[0:int(0.7 * len(outputs_norm)), -n_data_after:]
output_test = data_norm.iloc[int(0.7 * len(outputs_norm)):-1, -n_data_after:] 

In [40]:
entrada_train = tf.convert_to_tensor(input_train)
saida_train = tf.convert_to_tensor(output_train)
entrada_test = tf.convert_to_tensor(input_test)
saida_test = tf.convert_to_tensor(output_test)

In [41]:
## Montando a RNA

from keras.layers import SimpleRNN

In [42]:
## Criando a função do modelo da RNA

def create_model(inputs,outputs,n_neurons,activation_fn,type_nn):
  if type_nn == 'ff':
    model = keras.Sequential([
    keras.Input(shape=(len(inputs.columns),)),  # input layer (1)
    keras.layers.Dense(n_neurons, activation=activation_fn,
                      kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                      bias_regularizer=regularizers.l2(1e-4),
                      activity_regularizer=regularizers.l2(1e-5)),  # hidden layer (1)
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(n_neurons/2, activation=activation_fn,
                      kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                      bias_regularizer=regularizers.l2(1e-4),
                      activity_regularizer=regularizers.l2(1e-5)),  # hidden layer (2)
    
    # keras.layers.Dense(n_neurons, activation=activation_fn,
    #                    kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    #                    bias_regularizer=regularizers.l2(1e-4),
    #                    activity_regularizer=regularizers.l2(1e-5)),  # hidden layer (3)
                            
          #keras.layers.Dropout(0.2),
    keras.layers.Dense(len(outputs.columns), activation=activation_fn) # output layer 
    ])
  if type_nn == 'lstm':
    model = keras.Sequential()
    model.add(SimpleRNN(128,input_shape = (inputs.shape[1],inputs.shape[2]),return_sequences=True))
    model.add(Dense(1))


  return model

In [43]:
## Executando a RNA

n_neurons = 100
activation_fn = 'tanh'
type_nn = 'ff'

if type_nn == 'lstm':
  inputs = np.reshape(entrada_train, (entrada_train.shape[0],entrada_train.shape[1],1))
  outputs = np.reshape(saida_train, (saida_train.shape[0],saida_train.shape[1],1))
  model = create_model(inputs,outputs,n_neurons,activation_fn,type_nn)

if type_nn == 'ff':
  model = create_model(input_train,output_train,n_neurons,activation_fn,type_nn)

model.compile(optimizer ='adam',
              loss='mse',
              metrics=['mae','accuracy'])

model.summary()

early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',mode='min',patience = 10)

historico = model.fit(entrada_train, saida_train, shuffle=True, epochs=500,
                      validation_data=(input_test, output_test),callbacks=early_stop)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               3500      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 8601 (33.60 KB)
Trainable params: 8601 (33.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500
21/21 [==============================] - 2s 23ms/step - loss: 0.1694 - mae: 0.2811 - accuracy: 0.0015 - val_loss: 0.0969 - val_mae: 0.2389 - val_accuracy: 0.0000e+00
Epoch 2/500
21/21 [==============================] - 0s 5ms/step - loss: 0.1003 - mae: 0.2407 - accuracy: 0.

In [58]:
predictions = model.predict(entrada_test)

trace1 = go.Scatter(x = np.arange(0, len(saida_test)), # eixo x = meses da lista 'date
                    y = predictions[:, 0], # eixo y = temperatura máxima
                    mode = 'markers+lines', # tipo do gráfico: marcadorms em linhas
                    name = 'Dado Estimado') # legenda do objeto trace: T.MAX_INMET

trace2 = go.Scatter(x = np.arange(0, len(saida_test)), # eixo x = meses da lista 'date
                    y = saida_test[:, 0], # eixo y = temperatura máxima
                    mode = 'markers+lines', # tipo do gráfico: marcadorms em linhas
                    name = 'Dado Real') # legenda do objeto trace: T.MAX_INMET

# cria uma lista com os trace's
data_trace = [trace1, trace2]

# cria um objeto layout para por em um gráfico
# objeto layout: informar o que está sendo apresentado no gráfico, no caso abaixo, no eixo y é a 'Temperatura °C'
# no eixo x é informado que fica vazio, com o título do gráfico em caixa alta - comando .upper()
# alterando as características da fonte e cor de cada trace do gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Energia'},xaxis={'title': ''}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#B22222', '#006400', '#00BFFF', '#00FF7F', '#000080', '#FF0000' ])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data_trace, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%Y',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 6.1 - Comparativo de dados reais e estimados normalizados",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix=" Wh") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show(config={'scrollZoom': True})

9/9 [==============================] - 0s 2ms/step


In [57]:
pred_data = Denormalize(predictions,outputs['Energia (Wh) (k+1)'])
real_data = np.array(Denormalize(output_test,outputs))

trace1 = go.Scatter(x = np.arange(0, len(output_test)), # eixo x = meses da lista 'date
                    y = pred_data[:, 0], # eixo y = temperatura máxima
                    mode = 'markers+lines', # tipo do gráfico: marcadorms em linhas
                    name = 'Dado Estimado') # legenda do objeto trace: T.MAX_INMET

trace2 = go.Scatter(x = np.arange(0, len(saida_test)), # eixo x = meses da lista 'date
                    y = real_data[:, 0], # eixo y = temperatura máxima
                    mode = 'markers+lines', # tipo do gráfico: marcadorms em linhas
                    name = 'Dado Real') # legenda do objeto trace: T.MAX_INMET

# cria uma lista com os trace's
data_trace = [trace1, trace2]

# cria um objeto layout para por em um gráfico
# objeto layout: informar o que está sendo apresentado no gráfico, no caso abaixo, no eixo y é a 'Temperatura °C'
# no eixo x é informado que fica vazio, com o título do gráfico em caixa alta - comando .upper()
# alterando as características da fonte e cor de cada trace do gráfico
layout = go.Layout(title="Teste",yaxis={'title':'Energia'},xaxis={'title': ''}, 
                    font = {'family': 'Rockwell','size': 16,'color': 'black'},
                    colorway = ['#FF0000', '#000080', '#B22222', '#006400', '#00BFFF', '#00FF7F'])

# cria um gráfico através do objeto figure com os dados da lista com os traces e o layout acima
fig = go.Figure(data=data_trace, layout=layout)

# atualiza o gráfico, objeto figure, informando uma alteração no layout, esta alteração muda o range do eixo X,
# o formato de como será exibido esses dados do eixo X, o tema ,a legenda e o título do gráfico.
# seleciona o mode hovermode.
fig.update_layout(xaxis_tickformat = '%Y',
                    template = 'plotly_white',
                    legend_title="",
                    legend_orientation="h",
                    legend={
                        'x': 0.05,
                        'y': -0.25,
                    },
                    title={
                        'text': "Figura 6.2 - Comparativo de dados reais e estimados desnormalizados",
                        'y':0.93,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    hovermode=("x unified"))
fig.update_yaxes(ticksuffix=" Wh") 
fig.update_xaxes(ticksuffix="")                     # selecionando um sufixo para os dados no eixo Y

# comando utilizado para adicionar uma imagem ao gráfico
fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/LinconRozendo/Imagens_u/master/Multi%20Mapa/cear.png",
        xref="paper", yref="paper",
        x=1.05, y=1.1,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
        )
    )
    
# retorna a exibição do gráfico criado
fig.show(config={'scrollZoom': True})